In [1]:
from numpy import sin, cos, tan, arccos, arctan, deg2rad, rad2deg, sqrt, pi

In [2]:
'''In Wellbore-47 at 4305 md, compute TVD, Displacement, Northing, Easting, DLS, Closure Azimuth(Caz) & Vertical 
Section(@ 60.91 degrees.)
'''

AZ_vs = 60.91
# @ 4275 md = 751.535
Desp_ant = 751.535
TVD_1 = 4194.37
North_1 = -416.1
East_1 = 17.38

# Estación 1:
md_1 = 4275
inc_1 = 38.85
azi_1 = 35.56

# Estación 2:
md_2 = 4305
inc_2 = 43.08
azi_2 = 37.16

MD = md_2 - md_1

In [3]:
# Convertir la inclinación y azimuth a radianes
I1 = deg2rad(inc_1)
I2 = deg2rad(inc_2)
A1 = deg2rad(azi_1)
A2 = deg2rad(azi_2)

### Compute the partial displacement:

### $D_Parc = (MD_2 - MD_1) * sin(I_2)$

In [5]:
# Desplazamiento parcial
Desp_parc = MD * sin(I2)
Desp_tot = Desp_ant + Desp_parc
print(Desp_parc)
print(Desp_tot)

20.490565720276166
772.0255657202762


### Compute Dog Leg Severity(°/30 m)

### $ DLS = acos[(cosI_1 \cdot cosI_2) + (sinI_1 \cdot sinI_2) \cdot cos(A_2- A_1)] \cdot (\frac{30}{MD})$

In [6]:
DLS_rad =  (arccos((cos(I1) * cos(I2) + (sin(I1) * sin(I2) * cos(A2-A1))))) * (30/MD)
DLS_deg = rad2deg(DLS_rad)
DLS_deg

4.357829678415553

### Compute RF Factor:

### $RF = \frac{180}{\pi} \cdot \frac{2}{DLS} \cdot tan(\frac{DLS}{2})$

In [7]:
if DLS_deg == 0:
  RF = 1
else:
  RF = (180/pi) * (2/DLS_deg) * tan(deg2rad(DLS_deg/2))  
print(RF)

1.0004823533540457


### Compute True Vertical Depth (TVD)


$TVD_2 = [(\frac{MD_2-MD_1}{2}) \cdot (cosI_1 + cosI_2) \cdot RF] + TVD_1    $

In [8]:
TVD_2 = ((MD/2) * (cos(I1) + cos(I2)) * RF) + TVD_1
print(TVD_2)

4217.018793348547


### Compute Northing: 

$ North_2 = [(\frac{MD_2 - MD_1}{2}) \cdot ((sinI_1cos_A1) + (sinI_2cosA_2)) \cdot RF] + North_1 $

In [9]:
North_2 = ((MD/2) * (sin(I1) * cos(A1) + sin(I2) * cos(A2)) * RF) + North_1
print(North_2)

-400.2728781327542


### Compute Easting:

$ East_2 = [(\frac{MD_2 - MD_1}{2}) \cdot ((sinI_1sinA_1) + (sinI_2sinA_2)) \cdot RF] + East_1 $

In [10]:
East_2 = ((MD/2) * (sin(I1) * sin(A1) + sin(I2) * sin(A2)) * RF) + East_1
print(East_2)

29.046215754203104


### Compute Closure Distance - CD

$ CD =  \sqrt{(East^2) + (North^2)} $

In [12]:
CD = sqrt((East_2)**2 + (North_2)**2)
CD

401.32537873690273

### Compute closure azimuth (AZcl)


$ 0 - 90 ° : AZ_{CL} = arctan(\frac{East}{North}) $  
<br>
$ 90 - 180 ° : AZ_{CL} = 180 - arctan(\frac{East}{North}) $  
<br> 
$ 180 - 270 ° : AZ_{CL} = 180 + arctan(\frac{East}{North}) $  
<br>
$ 270 - 360 ° : AZ_{CL} = 360 - arctan(\frac{East}{North}) $  

In [29]:
# Primer cuadrante
if North_2 > 0 and East_2 > 0:
  AZ_cl = rad2deg(arctan(abs(East_2/North_2)))

# Segundo cuadrante
elif North_2 < 0 and East_2 > 0:
  AZ_cl = 180 - rad2deg(arctan(abs(East_2/North_2)))

# Tercer cuadrante
elif North_2 < 0 and East_2 < 0:
  AZ_cl = 180 + rad2deg(arctan(abs(East_2/North_2)))

# Cuarto cuadrante
else:
  AZ_cl = 360 - rad2deg(arctan(abs(East_2/North_2)))

AZ_cl

175.84954743836389

### Compute Vertical Section (Vs)

$ V_s = cos(AZ_{VS} - AZ_{CL}) \cdot CD $

In [30]:
Vs = cos(deg2rad(AZ_vs - AZ_cl)) * CD
print(Vs)

-169.22357546080357
